# Pydantic-based Mesh Example

This notebook demonstrates how to use the new Pydantic-based Mesh class in meshly. It covers:

1. Creating custom Mesh subclasses with additional attributes
2. Working with numpy arrays in Pydantic models
3. Encoding and decoding meshes to/from zip files
4. Optimizing meshes with the built-in optimization methods

In [1]:
import os
import numpy as np
from typing import Optional, List
from pydantic import Field

# Import the Mesh class
from meshly import Mesh

## 1. Creating a Custom Mesh Subclass

One of the key benefits of the new Pydantic-based Mesh class is the ability to create custom subclasses with additional attributes. Let's create a `TexturedMesh` class that adds texture coordinates and normals.

In [2]:
from pydantic import BaseModel, ConfigDict
from meshly import Packable

class MaterialProperties(BaseModel):
    """Material properties with numpy arrays - demonstrates BaseModel in dict edge case."""
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    name: str = Field(..., description="Material name")
    diffuse: np.ndarray = Field(..., description="Diffuse color array")
    specular: np.ndarray = Field(..., description="Specular color array")
    shininess: float = Field(32.0, description="Shininess value")


class PhysicsProperties(Packable):
    """Physics properties as a nested Packable - demonstrates Packable field support."""
    mass: float = Field(1.0, description="Object mass")
    friction: float = Field(0.5, description="Friction coefficient")
    # Arrays in nested Packable are encoded/decoded using the Packable's own encode/decode
    inertia_tensor: np.ndarray = Field(..., description="3x3 inertia tensor")
    collision_points: np.ndarray = Field(..., description="Collision sample points")


class TexturedMesh(Mesh):
    """
    A mesh with texture coordinates and normals.
    
    This demonstrates how to create a custom Mesh subclass with additional
    numpy array attributes that will be automatically encoded/decoded.
    """
    # Add texture coordinates and normals as additional numpy arrays
    texture_coords: np.ndarray = Field(..., description="Texture coordinates")
    normals: Optional[np.ndarray] = Field(None, description="Vertex normals")
    
    # Add non-array attributes
    material_name: str = Field("default", description="Material name")
    tags: List[str] = Field(default_factory=list, description="Tags for the mesh")

    # Dictionary containing nested dictionaries with arrays
    material_data: dict[str, dict[str, np.ndarray]] = Field(
        default_factory=dict,
        description="Nested dictionary structure with arrays"
    )

    material_colors: dict[str, str] = Field(
        default_factory=dict,
        description="Dictionary with non-array values"
    )

    # Dictionary containing BaseModel instances with numpy arrays
    # This demonstrates the edge case of dict[str, BaseModel] where BaseModel has arrays
    materials: dict[str, MaterialProperties] = Field(
        default_factory=dict,
        description="Dictionary of material name to MaterialProperties (BaseModel with arrays)"
    )
    
    # Nested Packable field - uses its own encode/decode methods
    # This demonstrates automatic handling of Packable fields within other Packables
    physics: Optional[PhysicsProperties] = Field(
        None,
        description="Physics properties as a nested Packable"
    )

## 2. Creating a Mesh Instance

Now let's create a simple cube mesh with texture coordinates and normals.

In [3]:
# Create vertices for a cube
vertices = np.array([
    [-0.5, -0.5, -0.5],  # 0: bottom-left-back
    [0.5, -0.5, -0.5],   # 1: bottom-right-back
    [0.5, 0.5, -0.5],    # 2: top-right-back
    [-0.5, 0.5, -0.5],   # 3: top-left-back
    [-0.5, -0.5, 0.5],   # 4: bottom-left-front
    [0.5, -0.5, 0.5],    # 5: bottom-right-front
    [0.5, 0.5, 0.5],     # 6: top-right-front
    [-0.5, 0.5, 0.5]     # 7: top-left-front
], dtype=np.float32)

# Create indices for the cube
indices = np.array([
    [0, 1, 2, 2, 3, 0],  # back face
    [1, 5, 6, 6, 2, 1],  # right face
    [5, 4, 7, 7, 6, 5],  # front face
    [4, 0, 3, 3, 7, 4],  # left face
    [3, 2, 6, 6, 7, 3],  # top face
    [4, 5, 1, 1, 0, 4]   # bottom face
], dtype=np.uint32)

# Create texture coordinates (one for each vertex)
texture_coords = np.array([
    [0.0, 0.0],  # 0
    [1.0, 0.0],  # 1
    [1.0, 1.0],  # 2
    [0.0, 1.0],  # 3
    [0.0, 0.0],  # 4
    [1.0, 0.0],  # 5
    [1.0, 1.0],  # 6
    [0.0, 1.0]   # 7
], dtype=np.float32)

# Create normals (one for each vertex)
normals = np.array([
    [0.0, 0.0, -1.0],  # 0: back
    [0.0, 0.0, -1.0],  # 1: back
    [0.0, 0.0, -1.0],  # 2: back
    [0.0, 0.0, -1.0],  # 3: back
    [0.0, 0.0, 1.0],   # 4: front
    [0.0, 0.0, 1.0],   # 5: front
    [0.0, 0.0, 1.0],   # 6: front
    [0.0, 0.0, 1.0]    # 7: front
], dtype=np.float32)

# Create MaterialProperties instances (BaseModel with numpy arrays)
cube_material = MaterialProperties(
    name="cube_material",
    diffuse=np.array([1.0, 0.5, 0.31], dtype=np.float32),
    specular=np.array([0.5, 0.5, 0.5], dtype=np.float32),
    shininess=32.0
)

secondary_material = MaterialProperties(
    name="secondary_material",
    diffuse=np.array([0.2, 0.8, 0.2], dtype=np.float32),
    specular=np.array([0.3, 0.3, 0.3], dtype=np.float32),
    shininess=16.0
)

# Create PhysicsProperties instance (nested Packable)
physics = PhysicsProperties(
    mass=2.5,
    friction=0.7,
    inertia_tensor=np.eye(3, dtype=np.float32) * 0.1,  # 3x3 identity scaled
    collision_points=np.array([
        [-0.5, -0.5, -0.5],
        [0.5, 0.5, 0.5],
        [0.0, 0.0, 0.0]
    ], dtype=np.float32)
)

# Create the textured mesh
mesh = TexturedMesh(
    vertices=vertices,
    indices=indices,
    texture_coords=texture_coords,
    normals=normals,
    material_name="cube_material",
    tags=["cube", "example"],
    material_data={
        "cube_material": {
            "diffuse": np.array([1.0, 0.5, 0.31], dtype=np.float32),
            "specular": np.array([0.5, 0.5, 0.5], dtype=np.float32),
            "shininess": np.array([32.0], dtype=np.float32)
        }
    },
    material_colors={
        "cube_material": "#FF7F50"
    },
    # dict[str, BaseModel] with numpy arrays inside
    materials={
        "cube_material": cube_material,
        "secondary_material": secondary_material
    },
    # Nested Packable field
    physics=physics
)

print(f"Mesh created with {mesh.vertex_count} vertices and {mesh.index_count} indices")
print(f"Material name: {mesh.material_name}")
print(f"Tags: {mesh.tags}")
print(f"Materials (BaseModel dict): {list(mesh.materials.keys())}")
print(f"Physics (nested Packable): mass={mesh.physics.mass}, friction={mesh.physics.friction}")

Mesh created with 8 vertices and 36 indices
Material name: cube_material
Tags: ['cube', 'example']
Materials (BaseModel dict): ['cube_material', 'secondary_material']
Physics (nested Packable): mass=2.5, friction=0.7


## 3. Optimizing the Mesh

The Mesh class provides several optimization methods that can be used to improve rendering performance.

In [4]:
# Optimize the mesh for vertex cache
vertex_cache_optimized_mesh = mesh.optimize_vertex_cache()
print("Optimized for vertex cache")

# Optimize the mesh for overdraw
overdraw_optimized_mesh = mesh.optimize_overdraw()
print("Optimized for overdraw")

# Optimize the mesh for vertex fetch
vertex_fetch_optimized = mesh.optimize_vertex_fetch()
print("Optimized for vertex fetch")

Optimized for vertex cache
Optimized for overdraw
Optimized for vertex fetch


## 4. Encoding and Saving the Mesh

The Mesh class provides methods for encoding the mesh and saving it to a zip file.

In [5]:
# Save the mesh to a zip file
zip_path = "textured_cube.zip"
mesh.save_to_zip(zip_path)
print(f"Saved mesh to {zip_path}, file size: {os.path.getsize(zip_path)} bytes")

Saved mesh to textured_cube.zip, file size: 7695 bytes


## 5. Loading the Mesh from a Zip File

The Mesh class provides a class method for loading a mesh from a zip file.

In [6]:
# Load the mesh from the zip file
loaded_mesh = TexturedMesh.load_from_zip(zip_path)
print(f"Loaded mesh: {loaded_mesh.vertex_count} vertices, {loaded_mesh.index_count} indices")
print(f"Material name: {loaded_mesh.material_name}")
print(f"Tags: {loaded_mesh.tags}")

# Verify that the texture coordinates and normals were loaded correctly
print(f"\nTexture coordinates shape: {loaded_mesh.texture_coords.shape}")
print(f"Normals shape: {loaded_mesh.normals.shape}")
print(f"Material data: {loaded_mesh.material_data}")
print(f"Material colors: {loaded_mesh.material_colors}")

# Verify the dict[str, BaseModel] edge case was loaded correctly
print(f"\n--- BaseModel dict edge case ---")
print(f"Materials keys: {list(loaded_mesh.materials.keys())}")
for mat_name, mat in loaded_mesh.materials.items():
    print(f"  {mat_name}:")
    print(f"    type: {type(mat).__name__}")
    print(f"    diffuse: {mat.diffuse}")
    print(f"    specular: {mat.specular}")
    print(f"    shininess: {mat.shininess}")

# Verify the nested Packable was loaded correctly
print(f"\n--- Nested Packable edge case ---")
print(f"Physics type: {type(loaded_mesh.physics).__name__}")
print(f"Physics mass: {loaded_mesh.physics.mass}")
print(f"Physics friction: {loaded_mesh.physics.friction}")
print(f"Physics inertia_tensor:\n{loaded_mesh.physics.inertia_tensor}")
print(f"Physics collision_points:\n{loaded_mesh.physics.collision_points}")

Loaded mesh: 8 vertices, 36 indices
Material name: cube_material
Tags: ['cube', 'example']

Texture coordinates shape: (8, 2)
Normals shape: (8, 3)
Material data: {'cube_material': {'diffuse': array([1.  , 0.5 , 0.31], dtype=float32), 'shininess': array([32.], dtype=float32), 'specular': array([0.5, 0.5, 0.5], dtype=float32)}}
Material colors: {'cube_material': '#FF7F50'}

--- BaseModel dict edge case ---
Materials keys: ['cube_material', 'secondary_material']
  cube_material:
    type: MaterialProperties
    diffuse: [1.   0.5  0.31]
    specular: [0.5 0.5 0.5]
    shininess: 32.0
  secondary_material:
    type: MaterialProperties
    diffuse: [0.2 0.8 0.2]
    specular: [0.3 0.3 0.3]
    shininess: 16.0

--- Nested Packable edge case ---
Physics type: PhysicsProperties
Physics mass: 2.5
Physics friction: 0.7
Physics inertia_tensor:
[[0.1 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  0.1]]
Physics collision_points:
[[-0.5 -0.5 -0.5]
 [ 0.5  0.5  0.5]
 [ 0.   0.   0. ]]


## 6. Creating a Different Mesh Subclass

Let's create another mesh subclass with different attributes to demonstrate the flexibility of the Pydantic-based Mesh class.

In [7]:
class SkinnedMesh(Mesh):
    """
    A mesh with skinning information for animation.
    """
    # Add bone weights and indices as additional numpy arrays
    bone_weights: np.ndarray = Field(..., description="Bone weights for each vertex")
    bone_indices: np.ndarray = Field(..., description="Bone indices for each vertex")
    
    # Add non-array attributes
    skeleton_name: str = Field("default", description="Skeleton name")
    animation_names: List[str] = Field(default_factory=list, description="Animation names")

# Create a simple skinned mesh
skinned_mesh = SkinnedMesh(
    vertices=vertices,
    indices=indices,
    bone_weights=np.random.random((len(vertices), 4)).astype(np.float32),  # 4 weights per vertex
    bone_indices=np.random.randint(0, 4, (len(vertices), 4)).astype(np.uint8),  # 4 bone indices per vertex
    skeleton_name="human_skeleton",
    animation_names=["walk", "run", "jump"]
)

print(f"Skinned mesh created with {skinned_mesh.vertex_count} vertices and {skinned_mesh.index_count} indices")
print(f"Skeleton name: {skinned_mesh.skeleton_name}")
print(f"Animation names: {skinned_mesh.animation_names}")
print(f"Bone weights shape: {skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {skinned_mesh.bone_indices.shape}")

Skinned mesh created with 8 vertices and 36 indices
Skeleton name: human_skeleton
Animation names: ['walk', 'run', 'jump']
Bone weights shape: (8, 4)
Bone indices shape: (8, 4)


## 7. Saving and Loading the Skinned Mesh

Let's save and load the skinned mesh to demonstrate that all attributes are preserved.

In [8]:
# Save the skinned mesh to a zip file
skinned_zip_path = "skinned_cube.zip"
skinned_mesh.save_to_zip(skinned_zip_path)
print(f"Saved skinned mesh to {skinned_zip_path}, file size: {os.path.getsize(skinned_zip_path)} bytes")

# Load the skinned mesh from the zip file
loaded_skinned_mesh = SkinnedMesh.load_from_zip(skinned_zip_path)
print(f"\nLoaded skinned mesh: {loaded_skinned_mesh.vertex_count} vertices, {loaded_skinned_mesh.index_count} indices")
print(f"Skeleton name: {loaded_skinned_mesh.skeleton_name}")
print(f"Animation names: {loaded_skinned_mesh.animation_names}")
print(f"Bone weights shape: {loaded_skinned_mesh.bone_weights.shape}")
print(f"Bone indices shape: {loaded_skinned_mesh.bone_indices.shape}")

Saved skinned mesh to skinned_cube.zip, file size: 2562 bytes

Loaded skinned mesh: 8 vertices, 36 indices
Skeleton name: human_skeleton
Animation names: ['walk', 'run', 'jump']
Bone weights shape: (8, 4)
Bone indices shape: (8, 4)


## 8. Cleaning Up (Part 1)

Let's clean up the files we created so far before the cache examples.

In [9]:
# Clean up
for path in [zip_path, skinned_zip_path]:
    if os.path.exists(path):
        os.remove(path)
        print(f"Removed {path}")

print("\nExample completed successfully!")

Removed textured_cube.zip
Removed skinned_cube.zip

Example completed successfully!


## 9. Using Cache for Nested Packables

When working with meshes that contain nested Packables (like our `TexturedMesh` with `PhysicsProperties`), you can use caching to deduplicate shared data and reduce file sizes. The cache uses SHA256 hashes for content-addressable storage.

In [10]:
import tempfile
from meshly import ReadHandler, WriteHandler

# Create a temporary cache directory
with tempfile.TemporaryDirectory() as cache_dir:
    print(f"Cache directory: {cache_dir}")
    
    # Create cache saver and loader using the handler factory methods
    cache_saver = WriteHandler.create_cache_saver(cache_dir)
    cache_loader = ReadHandler.create_cache_loader(cache_dir)
    
    # Save the mesh with caching - nested PhysicsProperties will be cached separately
    cached_zip_path = "textured_cube_cached.zip"
    mesh.save_to_zip(cached_zip_path, cache_saver=cache_saver)
    
    # Check what was cached
    import os
    cache_files = os.listdir(cache_dir)
    print(f"\nCached files ({len(cache_files)} items):")
    for f in cache_files:
        file_path = os.path.join(cache_dir, f)
        print(f"  {f}: {os.path.getsize(file_path)} bytes")
    
    # Compare file sizes
    original_size = os.path.getsize(zip_path) if os.path.exists(zip_path) else 0
    cached_size = os.path.getsize(cached_zip_path)
    print(f"\nOriginal zip size: {original_size} bytes")
    print(f"Cached zip size: {cached_size} bytes")
    
    # Load the mesh back using the cache
    loaded_cached_mesh = TexturedMesh.load_from_zip(cached_zip_path, cache_loader=cache_loader)
    
    # Verify the nested Packable was loaded correctly from cache
    print(f"\n--- Loaded from cache ---")
    print(f"Physics type: {type(loaded_cached_mesh.physics).__name__}")
    print(f"Physics mass: {loaded_cached_mesh.physics.mass}")
    print(f"Physics friction: {loaded_cached_mesh.physics.friction}")
    print(f"Physics inertia_tensor:\n{loaded_cached_mesh.physics.inertia_tensor}")
    
    # Clean up
    if os.path.exists(cached_zip_path):
        os.remove(cached_zip_path)
        print(f"\nRemoved {cached_zip_path}")

Cache directory: /tmp/tmpkflwgz8v

Cached files (1 items):
  210dc1059e9d5af349f0dad45dbbdc8797eb82b49e7a3443528337e33ce60854.zip: 1157 bytes

Original zip size: 0 bytes
Cached zip size: 6505 bytes

--- Loaded from cache ---
Physics type: PhysicsProperties
Physics mass: 2.5
Physics friction: 0.7
Physics inertia_tensor:
[[0.1 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  0.1]]

Removed textured_cube_cached.zip


### Cache Deduplication Example

When multiple meshes share the same nested Packable data, the cache automatically deduplicates them using SHA256 hashes.

In [ ]:
# Demonstrate cache deduplication - two meshes with identical physics properties
with tempfile.TemporaryDirectory() as cache_dir:
    print(f"\nCache directory for deduplication: {cache_dir}")
    cache_saver = WriteHandler.create_cache_saver(cache_dir)
    cache_loader = ReadHandler.create_cache_loader(cache_dir)
    
    # Create two meshes with identical physics (will share cache entry)
    shared_physics = PhysicsProperties(
        mass=1.0,
        friction=0.5,
        inertia_tensor=np.eye(3, dtype=np.float32),
        collision_points=np.array([[0, 0, 0]], dtype=np.float32)
    )
    
    mesh1 = TexturedMesh(
        vertices=vertices,
        indices=indices,
        texture_coords=texture_coords,
        normals=normals,
        material_name="mesh1",
        physics=shared_physics
    )
    
    mesh2 = TexturedMesh(
        vertices=vertices * 2,  # Different vertices
        indices=indices,
        texture_coords=texture_coords,
        normals=normals,
        material_name="mesh2",
        physics=shared_physics  # Same physics - will be deduplicated!
    )
    
    # Save both meshes with the same cache
    mesh1.save_to_zip("mesh1.zip", cache_saver=cache_saver)
    mesh2.save_to_zip("mesh2.zip", cache_saver=cache_saver)
    
    # Check the cache - should only have 1 entry (shared physics)
    cache_files = os.listdir(cache_dir)
    print(f"Cache entries: {len(cache_files)} (both meshes share the same physics cache)")
    
    # Load both meshes
    loaded1 = TexturedMesh.load_from_zip("mesh1.zip", cache_loader=cache_loader)
    loaded2 = TexturedMesh.load_from_zip("mesh2.zip", cache_loader=cache_loader)
    
    print(f"\nMesh1 material: {loaded1.material_name}, physics mass: {loaded1.physics.mass}")
    print(f"Mesh2 material: {loaded2.material_name}, physics mass: {loaded2.physics.mass}")
    
    # Clean up
    for f in ["mesh1.zip", "mesh2.zip"]:
        if os.path.exists(f):
            os.remove(f)


Cache directory for deduplication: /workspaces/meshly/cache
Cache entries: 1 (both meshes share the same physics cache)

Mesh1 material: mesh1, physics mass: 1.0
Mesh2 material: mesh2, physics mass: 1.0


## 10. Jax Conversion Example

In [12]:
try:
    import jax

    jax_skinned_mesh = skinned_mesh.convert_to("jax")
    print(f"Converted skinned mesh to JAX arrays, vertex dtype: {jax_skinned_mesh.vertices.dtype}")
except ImportError:
    print("JAX not available - skipping conversion example")

Converted skinned mesh to JAX arrays, vertex dtype: float32


## Example Complete!

This notebook demonstrated:
- Creating custom Mesh subclasses with additional numpy arrays
- Working with nested dictionaries containing arrays
- Using BaseModel instances with arrays inside dictionaries
- **Nested Packables** - fields that are themselves Packable classes
- **Cache support** - using `WriteHandler.create_cache_saver()` and `ReadHandler.create_cache_loader()` for content-addressable storage
- **Deduplication** - identical nested Packables share the same cache entry